In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

#Needed for training and evaluation
from losses import *
from RandomMatrixDataSet import get_sample,RandomMatrixDataSet,SingularvalueMatrix, EigenMatrix
from train import train_on_batch, run_training
from evaluation import *
from plotting import plot_loss_logs, error_histogram, plot_mean_identity_approx

#Seed and looks
torch.random.seed = 1234
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

In [ ]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        #This is what I added in terms of decoder 
        decoder_layers = TransformerDecoderLayer(d_model,nhead)
        self.transformer_decoder = TransformerDecoder(decoder_layers,nhead)
        
        #This used to be nn.Embedding
        self.embedding = MatrixEmbedding(d_model) 
        
        
        self.d_model = d_model
        self.generator = nn.Linear(d_model, ntoken) 

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        #self.encoder.weight.data.uniform_(-initrange, initrange)
        #self.generator.bias.data.zero_()
        #self.generator.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor,tgt: Tensor, tgt_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src_embedded = self.embedding(src) * math.sqrt(self.d_model)
        encoded_src = self.transformer_encoder(self.pos_encoder(src_embedded), src_mask)
        #Added this for the decoder  
        tgt_embedded = self.embedding(tgt)*math.sqrt(self.d_model) #
        output = self.transformer_decoder(self.pos_encoder(tgt_embedded),encoded_src,src_mask,tgt_mask)
        
        return self.generator(encoded_src) #This is without the decoder
        #return self.generator(output) This is with decoder


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
class MatrixEmbedding(nn.Module):
    """ """
    def __init__(self,d_model: int):
        super().__init__()
        self.d_model = d_model 
        #Maybe add a e.g. trainable linear embedding? 
        self.linear = nn.Linear(d_model,d_model)
        
    def forward(self, x: Tensor):
        x = x[:,:,None]
        m = torch.arange(x.shape[0]) 
        return torch.cat((x,m[:,None,None]), 2) #embed with index. Only works with d_model = 2
        #return torch.cat(self.d_model*[x], 2) #Just stack the tensor

### Here I generate a src-tgt pair to validate the pipe-line

In [ ]:
N = 1 
matrix_dimension = 3
matrix_parameters = {"N": N,
                    "d": matrix_dimension}
# Need to generate matrix data. These are just random matrices 
def get_batch(matrix_parameters):
    M = get_sample(matrix_parameters)
    M.compute_labels()
    data = torch.permute(torch.flatten(M.X, start_dim = 1),(1,0))
    target = torch.permute(torch.flatten(M.Y, start_dim = 1),(1,0))
    return data, target
                         
d,t = get_batch(matrix_parameters)

## Initiate an instance

In [ ]:
ntokens = 1 #size of "vocabulary"
emsize = 2  # embedding dimension - currently just adding index in sequence 
d_hid = 32  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout)#.to(device)
#Some normalization found from another source
#for p in model.parameters():
#    if p.dim() > 1:
#        nn.init.xavier_uniform_(p)

In [ ]:
import copy
import time

#Data parameters (Not using at the moment)
matrix_dimension = 3
batch_size = 10
matrix_parameters = {"N":batch_size,
                  "d": matrix_dimension}


#I am not entirely sure about the masks
src_mask = generate_square_subsequent_mask(matrix_dimension**2) #A fixed matrix size to begin with
tgt_mask = generate_square_subsequent_mask(matrix_dimension**2) # -||- 

#Training parameters
k = 10000

#criterion = nn.MSELoss()

#This is the loss Ive used in previous training
def relative_inv_MSE(predicted,x):
    #Normalize with batch square mean?
    id = torch.eye(x.shape[1])
    id_approx = torch.matmul(predicted,x)
    return((id_approx - id).square().mean()#/id_approx.square().mean()).mean()  #((id_approx - id).square()/x.square().mean()).mean()#

criterion = relative_inv_MSE
lr = 1e-2# learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(matrix_dimension**2).to(device)

    #num_batches = len(train_data) // bptt
    for i in range(k):
        #Currently just trying to overfit it on one matrix 
        data, targets = d,t #get_batch(matrix_parameters)
        
        #This is from the tutorial 
        #batch_size = data.size(0) 
        #if batch_size != bptt:  # only on last batch
        #    src_mask = src_mask[:batch_size, :batch_size]
        
        output = model(data, src_mask, targets, tgt_mask)
        #If we want to use (Xf(X)- 1) loss 
        loss = criterion(data.reshape(-1,matrix_dimension,matrix_dimension), output.reshape(-1,matrix_dimension,matrix_dimension))
        #For trying other loss functions
        #loss = criterion(output, targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        
        if i % log_interval == 0 and i > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            #ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {i:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f}')
            # print(f'| epoch {epoch:3d} | {i:5d} batches | '
            #      f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
            #      f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()
           
#Not using this at the moment
def evaluate(model: nn.Module, eval_parameters) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(matrix_dimension**2).to(device)
    with torch.no_grad():
        data, targets = get_batch(eval_parameters)
            #batch_size = data.size(0)
            #if batch_size != bptt:
            #    src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        output_flat = output.view(-1, ntokens)
        total_loss += batch_size * criterion(output_flat, targets.reshape(-1,ntokens)).item()
    return total_loss / (eval_parameters["N"] - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 1
best_model = None
eval_parameters = {"N": 100,
                   "d": matrix_dimension}
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    #val_loss = evaluate(model, eval_parameters)
    #val_ppl = math.exp(val_loss)
    #elapsed = time.time() - epoch_start_time
    #print('-' * 89)
    #print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
    #      f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    # print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = copy.deepcopy(model)

    scheduler.step()

In [ ]:
print(model(d,src_mask,t,tgt_mask)) #Just goes to some (or two) fixed values 
print(t)